# **Caso**

La planta PV Sol del Desierto, ubicada en María Elena (Región de Antofagasta, Chile), es un ejemplo de instalación fotovoltaica en un clima árido. Sus datos historicos de generación diaria, junto con registros meteorológicos locales (temperatura, viento, humedad, etc.), permite explorar cómo las condiciones climáticas influyen en la producción de energía solar. El reto es ajustar modelos estadísticos clasicos que, capten la mayor dinámica temporal y la heterocedasticidad propias de la serie de generación, y por otro, integren covariables meteorológicas para mejorar la calidad de los pronósticos.



# **Identificación y definición de las bases de datos**

In [ ]:
#Importamos librerias
import numpy as np
import pandas as pd
import seaborn as snsA
import matplotlib.pyplot as plta

import warnings
warnings.simplefilter("ignore")

In [ ]:
#Cargar los datos usando google.colab
from google.colab import files
uploaded = files.upload()

Saving Meteorologicas_220002_01012019_01072025_UTC_filted.xlsx to Meteorologicas_220002_01012019_01072025_UTC_filted.xlsx
Saving Generacion_Energia_Solar_Sol_Desierto_01012024_02072025_UTC.xlsx to Generacion_Energia_Solar_Sol_Desierto_01012024_02072025_UTC.xlsx


In [ ]:
df_meteorologicas = pd.read_excel(
    "Meteorologicas_220002_01012019_01072025_UTC_filted.xlsx"
)
df_meteorologicas.head(5)

,fecha_dia,temperatura_maxima,temperatura_promedio,temperatura_minima,cielo_despejado_proporcion,humedad_promedio,presion_promedio,rango_optico_meteorologico_promedio,direccion_viento_promedio,intensidad_viento_promedio,visibilidad_vertical_promedio
0,2019-01-01,25.770000,19.555455,13.033333,0.325000,25.503788,1014.774697,56416.666667,219.700000,13.007197,56515.151515
1,2019-01-02,27.001667,19.972500,13.683333,0.323485,28.212121,1014.970909,52475.000000,204.786364,13.374015,52522.727273
2,2019-01-03,24.626667,19.275530,13.841667,0.036364,27.587879,1015.106742,52856.060606,207.206818,11.957348,52787.878788
3,2019-01-04,24.525000,18.636742,8.816667,0.551515,22.781818,1016.911212,55121.212121,193.233333,13.101136,55079.545455
4,2019-01-05,24.963333,17.873636,9.816667,0.963636,24.590909,1019.478409,59535.606061,194.231818,12.803182,59628.787879


In [ ]:
# 1) Lee el archivo Excel
df_solar = pd.read_excel(
    "Generacion_Energia_Solar_Sol_Desierto_01012024_02072025_UTC.xlsx",
    sheet_name=0,               # o el nombre de la hoja si no es la primera
    engine='openpyxl'           # asegúrate de tener openpyxl instalado
)
df_solar.head(5)

,datetime,PFV SOL DEL DESIERTO
0,2024-01-01,1004.956955
1,2024-01-02,2050.680000
2,2024-01-03,2296.810000
3,2024-01-04,1657.720000
4,2024-01-05,2255.830000


## Identificación

In [ ]:
df_meteorologicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2374 entries, 0 to 2373
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   fecha_dia                            2374 non-null   datetime64[ns]
 1   temperatura_maxima                   2374 non-null   float64       
 2   temperatura_promedio                 2374 non-null   float64       
 3   temperatura_minima                   2374 non-null   float64       
 4   cielo_despejado_proporcion           2374 non-null   float64       
 5   humedad_promedio                     2374 non-null   float64       
 6   presion_promedio                     2374 non-null   float64       
 7   rango_optico_meteorologico_promedio  2374 non-null   float64       
 8   direccion_viento_promedio            2374 non-null   float64       
 9   intensidad_viento_promedio           2374 non-null   float64       
 10  visibilidad_

Identificamos 2374 observaciones no nulas en todas las variables, donde momento representa la fecha (por lo que es necesario definirla como objeto de tiempo). Además de esta, las demás variables, las cuales una vez vista su correcta definición como variables numericas se realizará un cambio de nombres para que su identificación sea más sencilla.

In [ ]:
df_solar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              549 non-null    datetime64[ns]
 1   PFV SOL DEL DESIERTO  549 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 8.7 KB


Aquí igualmente el tiempo debe definirse correctamente (datetime), mientras que PFV SOL DEL DESIERTO está bien definida como numerica, además con registros completos. Igualmente se cambiaran los nombres para mayor comprensión.

# **Transformación (renombrar variables)**

Las variables presentes en df_meteorologicas serán renombradas por las siguientes. (definimos como variable tiempo fecha_dia y eliminamos)

- momento -----> "fecha_dia"           
- isSkyClear ----> "cielo_despejado_proporcion"       
- hr ------------------> "humedad_promedio"                
- qff -----------------> "presion_promedio"              
- morInst ---------> "rango_optico_meteorologico_promedio"         
- ts_max -------------------> "temperatura_maxima"   
- ts_mean -------------------> "temperatura_promedio"  
- ts_min -------------------> "temperatura_minima"               
- ddInst ------------> "direccion_viento_promedio"          
- ffInst --------------> "intensidad_viento_promedio"          
- vis1Minuto -----> "visibilidad_vertical_promedio"     


In [ ]:
import pandas as pd

# 2) Renombrar columnas
rename_dict = {
    "momento":     "fecha_dia",
    "isSkyClear":  "cielo_despejado_proporcion",
    "hr":          "humedad_promedio",
    "qff":         "presion_promedio",
    "morInst":     "rango_optico_meteorologico_promedio",
    "ts_min":          "temperatura_minima",
    "ts_mean":          "temperatura_promedio",
    "ts_max":          "temperatura_maxima",
    "ddInst":      "direccion_viento_promedio",
    "ffInst":      "intensidad_viento_promedio",
    "vis1Minuto":  "visibilidad_vertical_promedio"
}
df_meteorologicas.rename(columns=rename_dict, inplace=True)

# 3) Fijar 'fecha_diaria' como índice
df_meteorologicas.set_index("fecha_dia", inplace=True)

# 5) Establecer frecuencia horaria
df_meteorologicas.index.freq = "D"




Las variables presentes en df_solar serán renombradas por las siguientes.

- datetime -------------------------> "fecha_dia"
- PFV SOL DEL DESIERTO  -> "generacion_MHw_sol_desierto"


In [ ]:
# 2) Renombrar las columnas según especificación
rename_dict = {
    "datetime":                     "fecha_dia",
    "PFV SOL DEL DESIERTO":         "generacion_MHw_sol_desierto"
}
df_solar.rename(columns=rename_dict, inplace=True)

# 3) Fijar 'fecha_dia' como índice datetime
df_solar.set_index("fecha_dia", inplace=True)


# 5) Establecer frecuencia horaria
df_solar.index.freq = "D"

# **Análisis exploratorio de los datos y elección de variables**

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1) Merge entre meteorológicas y solar
df = pd.merge(
    df_meteorologicas,
    df_solar,
    on='fecha_dia',
    how='inner'
)


# Lista de columnas numéricas
num_cols = df.select_dtypes(include='number').columns.tolist()

for col in num_cols:
    # Crear figura con 2 filas, 1 columna, eje x compartido
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.02,
        subplot_titles=(
            f"Boxplot de {col.replace('_',' ').title()}",
            f"Distribución de {col.replace('_',' ').title()}"
        )
    )

    # 1) Boxplot en la fila 1
    fig.add_trace(
        go.Box(x=df[col], name='', boxpoints='outliers'),
        row=1, col=1
    )

    # 2) Histograma (densidad) en la fila 2
    fig.add_trace(
        go.Histogram(
            x=df[col],
            histnorm='probability density',
            nbinsx=50,
            name=''
        ),
        row=2, col=1
    )

    # Layout general
    fig.update_layout(
        height=500,
        width=700,
        title_text=col.replace('_',' ').title(),
        title_x=0.5,
        showlegend=False,
        margin=dict(t=80, b=40, l=40, r=40)
    )

    fig.update_xaxes(title_text=col.replace('_',' ').title(), row=2, col=1)
    fig.update_yaxes(title_text='', row=1, col=1)
    fig.update_yaxes(title_text='Density', row=2, col=1)

    fig.show()


La temperatura máxima cuenta con un rango normal en el que se encuentra, esté entre 17.89°C y 28.7°C, donde en casos que se encuentran fuera de este rango normal se detectan mínimos y máximos de 9.6°C y 29.55°C, aunque son inusuales.

En la temperatura promedio se logra ver un sesgo hacia valores más altos de temperatura, donde su rango normal es entre 7.4°C y 21.41°C. Donde solo como casos inusuales son los casos de temperaturas más bajas, con un mínimo en 3.68°C.

Entre las 3 temperaturas analizadas se logra ver que la temperatura mínima es la que más puede estar siguiendo una distribución normal. Con un rango entre -3.4°C y 14.13°C. Con datos atípicos en la zona de temperaturas más bajas -4.89°C como mínimo. Esto sugiere que en las zonas, a pesar de analizar temperaturas mínimas, promedio y máximas, las temperaturas más bajas son casos inusuales en la zona.

Siguiendo con la línea anterior, se logra ver que al menos el 75% de los días presentan días mayormente despejados (más del 90% del día). Donde como casos inusuales son días con menos del 76% del día despejado.

En cuanto a la humedad promedio, su rango normal en el que se encuentra mayormente es entre 3.36% y 61.76%. Aunque al menos el 75% de los días se encuentran con una humedad promedio menor a 34%. Donde los casos atípicos en esta ocasión son días con humedad mayores a 62%.

Para la presión ocurre algo parecido, donde normalmente se encuentre en un rango entre 1016 hPa y 1034 hPa e inusualmente con presiones de hasta 1038.6 hPa como máximo.

Para el rango óptico meteorológico promedio se puede encontrar recurrentemente entre 41.23k metros y 66.2k metros, con valores inusuales en valores más bajos y una mediana en 53.93k metros.

La intensidad del viento igualmente parece ser de las distribuciones aparentemente más normales, con un rango recurrente entre 10.5 nudos y 16.27 nudos. Con un mínimo 8.42 y máximo 18.77 nudos como casos atípicos. (1 nudo = 1.852 km/h aprox).

La visibilidad promedio se puede encontrar entre 41.2k y 66.47k metros, y una mediana de 53.9k metros. Donde, para esta ocasión, los valores atípicos se encuentran en visibilidades más bajas que 41.2k metros como minino en 33k.

Por último, para la generación de energía solar en la planta de paneles fotovoltaicos Sol del Desierto en megawatts por día se logra ver que de forma habitual se encontrará entre 244.7 Mw y 2179.5 Mw por día. Por lo que, se logra notar que muestra un rendimiento constante en cuanto a la generación de energía, con casos atípicos en los cuales genera menos de 244.7 Mw, e igualmente en los cuales genera más que 2180 Mw por día.

In [ ]:
import plotly.express as px
# 1) Merge entre meteorológicas y solar
df = pd.merge(
    df_meteorologicas,
    df_solar,
    on='fecha_dia',
    how='inner'
)


# 1) Indica el nombre de la columna de generación en tu df
gen_col = "generacion_MHw_sol_desierto"  # p. ej. "generacion_MWh_sol_desierto"

# 2) Obtén todas las columnas numéricas excepto la de generación
clim_cols = [
    c for c in df.select_dtypes(include="number").columns
    if c != gen_col
]

# 3) Crea un scatter para cada variable climática
for clim in clim_cols:
    fig = px.scatter(
        df,
        x=clim,
        y=gen_col,
        trendline="ols",  # añade línea de regresión lineal
        labels={
            clim: clim.replace("_", " ").capitalize(),
            gen_col: "Generación (MWh)"
        },
        title=f"{clim.replace('_', ' ').capitalize()} vs Generación Solar"
    )
    # 4) Centrar el título
    fig.update_layout(title_x=0.5)
    fig.show()


Si visualizamos el comportamiento de la generación de energía solar con la temperatura mínima, se logra ver una pequeña relación positiva. Donde a mayores temperaturas mínimas sugieren mayores generaciones de energía solar. Lo mismo para la temperatura promedio y temperatura máxima. Por lo tanto, es posible que mayores temperaturas en general van de la mano con mayores generaciones de energía.

En cuanto a la proporción diaria de cielos despejados, se puede apreciar una gran concentración de días despejados (1). Sin embargo, viendo que días nublados (0) igual están asociados con una generación de energía solar dentro de los rangos normales. Este resultado posiblemente sesgado por la lejanía de la estación a la planta de paneles fotovoltaicos (70 km aproximadamente).

Ahora, por parte de la presión se observa una pequeña relación inversa con la generación de energía solar. Donde se propone que a medida que la presión es menor 2 existen mayores niveles de generación de energía. Lo mismo para el rango óptico meteorológico, dirección del viento y visibilidad vertical promedio. En cambio, no así con la intensidad del viento promedio, en donde no se logra ver una relación clara entre sí mismo y la generación de energía.

In [ ]:
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

# 1) Merge entre meteorológicas y solar
df = pd.merge(
    df_meteorologicas,
    df_solar,
    on='fecha_dia',
    how='inner'
)

# 2) Seleccionar solo las columnas numéricas
df_num = df.select_dtypes(include=['number'])
cols = df_num.columns

# 3) Calcular matrices de correlación y p-valor (Pearson)
corr = pd.DataFrame(index=cols, columns=cols, dtype=float)
pval = pd.DataFrame(index=cols, columns=cols, dtype=float)

for i in cols:
    for j in cols:
        r, p = stats.pearsonr(df_num[i], df_num[j])
        corr.at[i, j] = r
        pval.at[i, j] = p

# 4) Crear matriz de texto con el valor correlación y "*" si p < 0.05
annot = corr.round(2).astype(str)
mask_signif = pval < 0.05
annot = annot.where(~mask_signif, annot + '*')

# 5) Dibujar heatmap con Plotly
heatmap = go.Heatmap(
    z=corr.values,
    x=cols,
    y=cols,
    colorscale='RdBu',
    zmin=-1,
    zmax=1,
    showscale=True,
    colorbar=dict(title='Correlación')
)

fig = go.Figure(data=[heatmap])

# 6) Añadir anotaciones de texto sobre cada celda
for i, row in enumerate(cols):
    for j, col in enumerate(cols):
        fig.add_annotation(
            x=col, y=row,
            text=annot.at[row, col],
            showarrow=False,
            font=dict(size=10),
            xanchor='center',
            yanchor='middle'
        )

fig.update_layout(
    title='Heatmap de correlaciones (meteorología vs. generación solar)',
    title_x=0.5,
    xaxis_nticks=len(cols),
    yaxis_nticks=len(cols),
    width=900,
    height=900,
    margin=dict(l=120, r=120, t=120, b=120)
)

fig.show()



Para poder pronosticar con variables exógenas, es necesario primero predecir estas variables exógenas, donde la que presenta mayor correlación con la generación de energía solar es temperatura máxima (r=0.16 y significativa con un nivel de significación del 0.05). Esto con los datos en el mismo rango de tiempos (Ya que generación de energía mantiene datos desde 2024 y 2025. En cambio, las variables meteorologicas de 2019 a 2025). Ahora viendo el grafico de correlación para los datos completos de meteorologia.

In [ ]:
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

# 2) Seleccionar solo las columnas numéricas
df_num = df_meteorologicas.select_dtypes(include=['number'])
cols = df_num.columns

# 3) Calcular matrices de correlación y p-valor (Pearson)
corr = pd.DataFrame(index=cols, columns=cols, dtype=float)
pval = pd.DataFrame(index=cols, columns=cols, dtype=float)

for i in cols:
    for j in cols:
        r, p = stats.pearsonr(df_num[i], df_num[j])
        corr.at[i, j] = r
        pval.at[i, j] = p

# 4) Crear matriz de texto con el valor correlación y "*" si p < 0.05
annot = corr.round(2).astype(str)
mask_signif = pval < 0.05
annot = annot.where(~mask_signif, annot + '*')

# 5) Dibujar heatmap con Plotly
heatmap = go.Heatmap(
    z=corr.values,
    x=cols,
    y=cols,
    colorscale='RdBu',
    zmin=-1,
    zmax=1,
    showscale=True,
    colorbar=dict(title='Correlación')
)

fig = go.Figure(data=[heatmap])

# 6) Añadir anotaciones de texto sobre cada celda
for i, row in enumerate(cols):
    for j, col in enumerate(cols):
        fig.add_annotation(
            x=col, y=row,
            text=annot.at[row, col],
            showarrow=False,
            font=dict(size=10),
            xanchor='center',
            yanchor='middle'
        )

fig.update_layout(
    title='Heatmap de correlaciones (meteorología)',
    title_x=0.5,
    xaxis_nticks=len(cols),
    yaxis_nticks=len(cols),
    width=900,
    height=900,
    margin=dict(l=120, r=120, t=120, b=120)
)

fig.show()



 Ahora, viendo cuáles son las con mayor correlación con temperatura máxima son: temperatura_promedio, presion_promedio y temperatura_minima. Las cuales integraremos en un modelo VAR ajustadas como variables endogenas al presentar correlación entre ellas.

# **Modelado para variables climaticas**

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# 0) Asegurarnos de que el índice es datetime y se llama 'fecha_dia'
df_meteorologicas.index = pd.to_datetime(df_meteorologicas.index)
df_meteorologicas.index.rename('fecha_dia', inplace=True)

# 1) Seleccionar las series endógenas
endog_vars = [
    'temperatura_maxima',
    'presion_promedio',
    'temperatura_minima',
    'temperatura_promedio'
]
endog = df_meteorologicas[endog_vars]

# 2) Test ADF por cada serie
print("Resultados ADF Test:\n")
for col in endog_vars:
    series = endog[col].dropna()
    result = adfuller(series, autolag='AIC')
    adf_stat, p_value, used_lag, nobs, crit_vals, icbest = result

    print(f"{col}:")
    print(f"  – ADF statistic : {adf_stat:.4f}")
    print(f"  – p-value       : {p_value:.4f}")
    print("  – Critical values:")
    for cv1, cv2 in crit_vals.items():
        print(f"      {cv1}: {cv2:.4f}")
    print()

# 3) Interpretación básica
# Si p-value < 0.05 ⇒ rechazamos H0 (serie es estacionaria)


Resultados ADF Test:

temperatura_maxima:
  – ADF statistic : -3.5081
  – p-value       : 0.0078
  – Critical values:
      1%: -3.4331
      5%: -2.8628
      10%: -2.5674

presion_promedio:
  – ADF statistic : -3.6050
  – p-value       : 0.0057
  – Critical values:
      1%: -3.4331
      5%: -2.8628
      10%: -2.5674

temperatura_minima:
  – ADF statistic : -3.8354
  – p-value       : 0.0026
  – Critical values:
      1%: -3.4331
      5%: -2.8628
      10%: -2.5674

temperatura_promedio:
  – ADF statistic : -3.0714
  – p-value       : 0.0287
  – Critical values:
      1%: -3.4331
      5%: -2.8628
      10%: -2.5674



Podemos notar que con un nivel de significación del 0.05 se rechaza la hipotesis nula. Es decir, existe suficiente evidencia muestral a favor de que las series temperatura_promedio, temperatura_minima, presion_promedio y temperatura_maxima cumplen con estacionariedad. Por lo que decidimos d=0 para el varma a ajustar.

In [ ]:
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1) Preparar DataFrame
df = df_meteorologicas.copy()
if 'fecha_dia' in df.columns:
    df['fecha_dia'] = pd.to_datetime(df['fecha_dia'])
    df.set_index('fecha_dia', inplace=True)
df = df.sort_index().asfreq('D').fillna(method='ffill')

# 2) Definir variables endógenas
cols = [
    'temperatura_maxima',
    'presion_promedio',
    'temperatura_minima',
    'temperatura_promedio'
]
endog = df[cols]

# 3) Selección de p vía AIC
maxlags = 15
aic_vals = []
for p in range(1, maxlags + 1):
    res_p = VAR(endog).fit(p)
    aic_vals.append(res_p.aic)

aic_series = pd.Series(aic_vals, index=range(1, maxlags + 1))
print("AIC para cada orden p:\n", aic_series, "\n")

p_opt = aic_series.idxmin()
print(f"Orden óptimo p elegido (min AIC): {p_opt}\n")

# 4) Split 80 % entrenamiento / 20 % validación
n = len(endog)
n_train = int(n * 0.8)
train = endog.iloc[:n_train]
test  = endog.iloc[n_train:]

# 5) Ajustar VAR(p_opt) sobre entrenamiento
model_train = VAR(train)
res_train  = model_train.fit(p_opt)

# 6) Forecast sobre validación
lag_order      = res_train.k_ar
forecast_input = train.values[-lag_order:]
steps          = len(test)
fc_vals        = res_train.forecast(y=forecast_input, steps=steps)

df_fc_test = pd.DataFrame(fc_vals, index=test.index, columns=cols)

# 7) Gráfico de validación 2×2: real vs. predicho
fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=True,
    subplot_titles=[c.replace('_',' ').title() for c in cols],
    vertical_spacing=0.08,
    horizontal_spacing=0.08
)

for i, c in enumerate(cols):
    r = (i // 2) + 1
    cc = (i % 2) + 1

    # Serie real (20% de test)
    fig.add_trace(
        go.Scatter(
            x=test.index, y=test[c],
            mode='lines', name=f'Real {c}'
        ),
        row=r, col=cc
    )
    # Serie predicha
    fig.add_trace(
        go.Scatter(
            x=df_fc_test.index, y=df_fc_test[c],
            mode='lines',
            line=dict(dash='dash'),
            name=f'Predicho {c}'
        ),
        row=r, col=cc
    )

    fig.update_xaxes(title_text='Fecha', row=r, col=cc)
    fig.update_yaxes(title_text=c.replace('_',' ').title(), row=r, col=cc)

fig.update_layout(
    height=800, width=1000,
    title_text=f'VAR({p_opt}) Validación 20 %: Real vs Predicho',
    title_x=0.5,
    showlegend=False,
    margin=dict(t=100, b=50, l=50, r=50)
)

fig.show()


AIC para cada orden p:
 1     1.150559
2     0.955044
3     0.872710
4     0.854116
5     0.849323
6     0.844228
7     0.838078
8     0.839484
9     0.841778
10    0.841410
11    0.839388
12    0.846113
13    0.847121
14    0.844609
15    0.839084
dtype: float64 

Orden óptimo p elegido (min AIC): 7



El numero de autoregresivos a utilizar en este caso es 7 elegido por criterio de AIC, con el fin de elegir el menor AIC al penalizar numeros de autoregresivos grandes evitando la complejidad de este.

En cuanto al pronostico realizado y comparando con los valores observados en cada una, se logra ver que captura la tendencia existente y la sigue hasta 1 mes en adelante, pero luego estabilizandose en un valor aproximadamente constante.

Luego, ajustando el modelo VAR con el numero de autoregresivos (p=7) con los datos completos y pronosticando las variables endogenas a 30 días.

In [ ]:
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1) Preparar DataFrame
df = df_meteorologicas.copy()
if 'fecha_dia' in df.columns:
    df['fecha_dia'] = pd.to_datetime(df['fecha_dia'])
    df.set_index('fecha_dia', inplace=True)
df = df.sort_index().asfreq('D').fillna(method='ffill')

# 2) Definir variables endógenas
cols = [
    'temperatura_maxima',
    'presion_promedio',
    'temperatura_minima',
    'temperatura_promedio'
]
endog = df[cols]

# 3) Ajuste final VAR(p_opt) sobre TODO el histórico
p_opt = 7  # usa aquí el p que obtuviste en el paso 1
model_full = VAR(endog)
res_full   = model_full.fit(p_opt)

# 4) Forecast 30 días adelante
lag_order      = res_full.k_ar
forecast_input = endog.values[-lag_order:]
n_steps        = 30
fc_vals        = res_full.forecast(y=forecast_input, steps=n_steps)

idx_future = pd.date_range(
    start=endog.index[-1] + pd.Timedelta(days=1),
    periods=n_steps, freq='D'
)
df_fc_future = pd.DataFrame(fc_vals, index=idx_future, columns=cols)

# 5) Filtrar histórico desde octubre 2024
start_date = '2024-10-01'
hist_filtered = endog.loc[start_date:]

# 6) Gráfico 2×2: histórico filtrado vs. forecast
fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=True,
    subplot_titles=[c.replace('_',' ').title() for c in cols],
    vertical_spacing=0.08,
    horizontal_spacing=0.08
)

for i, c in enumerate(cols):
    r  = (i // 2) + 1
    cc = (i % 2) + 1

    # Serie histórica filtrada
    fig.add_trace(
        go.Scatter(
            x=hist_filtered.index, y=hist_filtered[c],
            mode='lines', name=f'Histórico {c}'
        ),
        row=r, col=cc
    )
    # Pronóstico 30 días
    fig.add_trace(
        go.Scatter(
            x=df_fc_future.index, y=df_fc_future[c],
            mode='lines', line=dict(dash='dash'),
            name=f'Forecast {c}'
        ),
        row=r, col=cc
    )

    fig.update_xaxes(title_text='Fecha', row=r, col=cc, range=[start_date, df_fc_future.index[-1]])
    fig.update_yaxes(title_text=c.replace('_',' ').title(), row=r, col=cc)

fig.update_layout(
    title_text=f'VAR({p_opt}) — Pronóstico 30 Días vs Histórico (desde {start_date})',
    title_x=0.5,
    height=800,
    width=1000,
    showlegend=False,
    margin=dict(t=100, b=50, l=50, r=50)
)

fig.show()


In [ ]:
# --- 1) Forecast 30 días adelante (ya lo tienes hecho) ---
lag_order      = res_full.k_ar
forecast_input = endog.values[-lag_order:]
n_steps        = 30
fc_vals        = res_full.forecast(y=forecast_input, steps=n_steps)

# --- 2) Crear índice de fechas futuras ---
idx_future = pd.date_range(
    start=endog.index[-1] + pd.Timedelta(days=1),
    periods=n_steps, freq='D'
)

# --- 3) Construir DataFrame con los pronósticos ---
df_pronosticos = pd.DataFrame(
    data=fc_vals,
    index=idx_future,
    columns=cols
)

Podemos nota que en esta ocasión al ser menos días a predecir (solo un mes) logra capturar la variabilidad de los días hasta finales de julio. Estos valores pronosticados serán guardados con el fin de aplicar un sarimax considerando la generación de energía solar como estacional.

Ahora revisamos sus metricas (RMSE y MAE).

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# 1) División out-of-sample de 30 días
horizon    = 30
train_end  = len(endog) - horizon
train_ds   = endog.iloc[:train_end]
test_ds    = endog.iloc[train_end:]

# 2) Ajuste VAR sobre train_ds
res_val    = VAR(train_ds).fit(p_opt)

# 3) Forecast sobre test_ds
lag_order      = res_val.k_ar
forecast_input = train_ds.values[-lag_order:]
fc_vals_val    = res_val.forecast(y=forecast_input, steps=horizon)
df_pred_val    = pd.DataFrame(fc_vals_val, index=test_ds.index, columns=cols)

# 4) Cálculo de métricas RMSE y MAE por variable
metrics = []
for c in cols:
    y_true = test_ds[c]
    y_pred = df_pred_val[c]
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    metrics.append({'Variable': c, 'RMSE': rmse, 'MAE': mae})

# 5) Mostrar tabla de métricas
metrics_df = pd.DataFrame(metrics)
print(metrics_df.to_string(index=False))


            Variable     RMSE      MAE
  temperatura_maxima 3.540192 2.484447
    presion_promedio 2.421273 1.981734
  temperatura_minima 3.424390 2.673421
temperatura_promedio 2.155991 1.815237


Un MAE (Error medio absoluto) de 1.81°C en temperatura promedio sugiere que en promedio se desvía 1.81°C la predicción del valor real. Un RMSE (raiz del error cuadratico medio) de 2.16°C (cercano al MAE) al ser mayor al MAE indica que algunos días se comenten errores altos (dado por posibles casos atípicos). Siendo el ajuste de temperatura promedio el que mejor se ajusta en cuanto a RMSE y MAE.

Para la presión promedio igualmente muestra un RMSE = 2.42 hPa y MAE = 1.98 hPa, también siendo el mejor ajuste entre los demás.

Para la temperatura maxima y minima dan errores algo mayores a los demás, lo cual es lógico al capturar el maximo diario que repesenta solo un momento dado; minutos o horas.

# **Modelado para generación de energía solar**

In [ ]:
df_solar

,generacion_MHw_sol_desierto
fecha_dia,
2024-01-01,1004.956955
2024-01-02,2050.680000
2024-01-03,2296.810000
2024-01-04,1657.720000
2024-01-05,2255.830000
...,...
2025-06-28,1180.510000
2025-06-29,1201.680000
2025-06-30,1231.170000


Con base en el menor AIC se encontrará la mejor combinación de parametros para el modelo que minimicen el criterio AIC (que penaliza parametros muy elevados).

In [ ]:
import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
warnings.filterwarnings("ignore")
# --- 0) Detectar columnas relevantes ---
# Encuentra la columna de generación en df_solar (que contenga 'generacion' en su nombre)
gen_col = next(c for c in df_solar.columns if 'generacion' in c.lower())
exo_col = 'temperatura_maxima'

# --- 1) Prepara índices datetime ---
df_s = df_solar.copy()
if not isinstance(df_s.index, pd.DatetimeIndex):
    if 'fecha_dia' in df_s.columns:
        df_s['fecha_dia'] = pd.to_datetime(df_s['fecha_dia'])
        df_s.set_index('fecha_dia', inplace=True)
    else:
        raise KeyError("df_solar no tiene índice datetime ni columna 'fecha_dia'")

df_m = df_meteorologicas.copy()
if not isinstance(df_m.index, pd.DatetimeIndex):
    if 'fecha_dia' in df_m.columns:
        df_m['fecha_dia'] = pd.to_datetime(df_m['fecha_dia'])
        df_m.set_index('fecha_dia', inplace=True)
    else:
        raise KeyError("df_meteorologicas no tiene índice datetime ni columna 'fecha_dia'")

# --- 2) Merge inner en fechas comunes ---
df = df_s[[gen_col]].join(df_m[[exo_col]], how='inner')

# --- 3) Define target y exógena ---
y    = df[gen_col]
exog = df[[exo_col]]

# --- 4) Grid‐search SARIMAX(p,d,q)(P,D,Q,7) ---
s = 7
p_rng = range(0, 3)
d_rng = range(0, 2)
q_rng = range(0, 3)
P_rng = range(0, 2)
D_rng = range(0, 2)
Q_rng = range(0, 2)

best_aic      = np.inf
best_order    = None
best_seasonal = None

for p, d, q in itertools.product(p_rng, d_rng, q_rng):
    for P, D, Q in itertools.product(P_rng, D_rng, Q_rng):
        try:
            m = SARIMAX(
                y,
                exog=exog,
                order=(p, d, q),
                seasonal_order=(P, D, Q, s),
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            r = m.fit(disp=False)
            if r.aic < best_aic:
                best_aic      = r.aic
                best_order    = (p, d, q)
                best_seasonal = (P, D, Q, s)
        except:
            continue

print(f"Mejor SARIMAX encontrado: order={best_order}, seasonal_order={best_seasonal}, AIC={best_aic:.2f}")



Mejor SARIMAX encontrado: order=(1, 1, 2), seasonal_order=(0, 1, 1, 7), AIC=7581.43


La mejor combinación de parametros para un sarimax resultó (con un AIC minimo) ser para SARIMAX(1,1,2)(0,1,1)[7] el cual entregó un AIC de 7581.43.

Entrenaremos con el 80% de los datos (de temperatura maxima y generacion de energia) para validar con el 20%.

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go
warnings.filterwarnings("ignore")

# --- Supuestos previos: ya tienes en el entorno ---
# df               : DataFrame con índice datetime y columnas ['generacion...', 'temperatura_maxima']
# best_order       : tu tupla óptima (p,d,q)
# best_seasonal    : tu tupla óptima (P,D,Q,s)

gen_col = next(c for c in df.columns if 'generacion' in c.lower())
exo_col = 'temperatura_maxima'

# 1) Split 80% / 20%
n        = len(df)
n_train  = int(n * 0.8)
train    = df.iloc[:n_train]
validate = df.iloc[n_train:]

# 2) Ajustar SARIMAX solo con entrenamiento
mod_val = SARIMAX(
    train[gen_col],
    exog=train[[exo_col]],
    order=best_order,
    seasonal_order=best_seasonal,
    enforce_stationarity=False,
    enforce_invertibility=False
)
res_val = mod_val.fit(disp=False)

# 3) Pronóstico sobre validación
steps = len(validate)
pred   = res_val.get_forecast(steps=steps, exog=validate[[exo_col]])
forecast_mean = pred.predicted_mean

# 4) Construir DataFrame comparativo
df_val = validate[[gen_col]].copy()
df_val['pred_generacion'] = forecast_mean.values

# 5) Métricas de validación
rmse = np.sqrt(mean_squared_error(df_val[gen_col], df_val['pred_generacion']))
mae  = mean_absolute_error(df_val[gen_col], df_val['pred_generacion'])
print(f"Validación SARIMAX: RMSE = {rmse:.3f}, MAE = {mae:.3f}")

# 6) Gráfico comparativo
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_val.index, y=df_val[gen_col],
    mode='lines', name='Real'
))
fig.add_trace(go.Scatter(
    x=df_val.index, y=df_val['pred_generacion'],
    mode='lines', line=dict(dash='dash'), name='Predicho'
))
fig.update_layout(
    title="Validación SARIMAX: Real vs Predicho",
    title_x=0.5,
    xaxis_title="Fecha",
    yaxis_title=gen_col.replace('_',' ').title(),
    width=800, height=400
)
fig.show()

# 7) El DataFrame final df_val incluye fecha, real y predicho:
print(df_val.head())


Validación SARIMAX: RMSE = 458.049, MAE = 374.315


            generacion_MHw_sol_desierto  pred_generacion
fecha_dia                                               
2025-03-14                      1252.79       715.625756
2025-03-15                      1178.41       925.077970
2025-03-16                       950.99       913.484752
2025-03-17                       867.23       955.575671
2025-03-18                      1375.90      1012.820775


Podemos ver en la validación que la variabilidad existente en la generación día con día no se logra capturar correctamente. El pronostico de los días en la validación mantiene variaciones, pero insuficientes en comparación a la generación de energía solar real. Esto puede sugerir la falta de variables explicativas que tengan mayor peso y relación con la generación de energía. Por lo que las variables climaticas (la más correlacionada; temperatura maxima) no son validas para acompañar y explicar la generación diaria de energía solar.

In [ ]:
import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go
warnings.filterwarnings("ignore")

# --- 0) Asegura índices datetime en ambos DataFrames ---
def ensure_datetime_index(df, col='fecha_dia'):
    df = df.copy()
    if not isinstance(df.index, pd.DatetimeIndex):
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        else:
            raise KeyError(f"{df.name} necesita columna '{col}' o índice datetime")
    return df.sort_index().asfreq('D').fillna(method='ffill')

df_s = ensure_datetime_index(df_solar)
df_s.name = 'df_solar'
df_m = ensure_datetime_index(df_meteorologicas)
df_m.name = 'df_meteorologicas'

# --- 1) Merge para entrenamiento: solo fechas comunes ---
# Detecta columnas
gen_col = next(c for c in df_s.columns if 'generacion' in c.lower())
exo_var = 'temperatura_maxima'

df_train = df_s[[gen_col]].join(df_m[[exo_var]], how='inner')

# --- 2) Grid‐search SARIMA para la exógena ---
temp = df_m[exo_var]  # serie completa de temperatura
s = 7
p_rng = range(0,3); d_rng = range(0,2); q_rng = range(0,3)
P_rng = D_rng = Q_rng = range(0,2)

best_aic_temp = np.inf
for p,d,q in itertools.product(p_rng,d_rng,q_rng):
    for P,D,Q in itertools.product(P_rng,D_rng,Q_rng):
        try:
            mod = SARIMAX(temp,
                          order=(p,d,q),
                          seasonal_order=(P,D,Q,s),
                          enforce_stationarity=False,
                          enforce_invertibility=False)
            res = mod.fit(disp=False)
            if res.aic < best_aic_temp:
                best_aic_temp = res.aic
                best_order_temp = (p,d,q)
                best_seasonal_temp = (P,D,Q,s)
        except:
            pass

# Ajuste final de temperatura
mod_temp = SARIMAX(temp,
                   order=best_order_temp,
                   seasonal_order=best_seasonal_temp,
                   enforce_stationarity=False,
                   enforce_invertibility=False)
res_temp = mod_temp.fit(disp=False)

# Forecast 30d de la exógena
n_steps = 30
fc_temp = res_temp.get_forecast(steps=n_steps)
idx_future = pd.date_range(start=temp.index[-1] + pd.Timedelta(days=1),
                           periods=n_steps, freq='D')
df_exog_fc = fc_temp.predicted_mean.rename(exo_var).to_frame()

# --- 3) Ajuste SARIMAX de generación sobre datos comunes ---
y     = df_train[gen_col]
exog_h= df_train[[exo_var]]

# Grid‐search SARIMAX(p,d,q)(P,D,Q,7) con exógena para generación
best_aic = np.inf
for p,d,q in itertools.product(p_rng,d_rng,q_rng):
    for P,D,Q in itertools.product(P_rng,D_rng,Q_rng):
        try:
            m = SARIMAX(y,
                        exog=exog_h,
                        order=(p,d,q),
                        seasonal_order=(P,D,Q,s),
                        enforce_stationarity=False,
                        enforce_invertibility=False)
            r = m.fit(disp=False)
            if r.aic < best_aic:
                best_aic     = r.aic
                best_order   = (p,d,q)
                best_seasonal= (P,D,Q,s)
        except:
            pass

# Ajuste final de generación
mod_gen = SARIMAX(y,
                  exog=exog_h,
                  order=best_order,
                  seasonal_order=best_seasonal,
                  enforce_stationarity=False,
                  enforce_invertibility=False)
res_gen = mod_gen.fit(disp=False)

# --- 4) Pronóstico generación 30d usando exógenas predichas ---
forecast = res_gen.get_forecast(steps=n_steps, exog=df_exog_fc)
pred_mean = forecast.predicted_mean
conf_int  = forecast.conf_int()

df_fc_gen = pd.DataFrame({
    'pred_generacion': pred_mean,
    'lower_ci':        conf_int.iloc[:, 0],
    'upper_ci':        conf_int.iloc[:, 1],
    exo_var:           df_exog_fc[exo_var]
}, index=idx_future)

# 5) Mostrar resultados y gráfico con 30 días de histórico y 30 días de pronóstico

# asume que ya tienes:
# y: pd.Series de generación histórica (entrenada sobre todo el histórico común)
# df_fc_gen: DataFrame con índice de 30 días futuros y columnas ['pred_generacion','lower_ci','upper_ci',exo_var]

import plotly.graph_objects as go

# Extrae los últimos 30 días de histórico
hist_last30 = y.iloc[-30:]

fig = go.Figure()

# Histórico (últimos 30 días)
fig.add_trace(go.Scatter(
    x=hist_last30.index, y=hist_last30.values,
    mode='lines', name='Histórico (últimos 30 días)'
))

# Banda de confianza del forecast
fig.add_trace(go.Scatter(
    x=df_fc_gen.index, y=df_fc_gen['upper_ci'],
    mode='lines', line=dict(color='lightgrey'), showlegend=False
))
fig.add_trace(go.Scatter(
    x=df_fc_gen.index, y=df_fc_gen['lower_ci'],
    mode='lines', fill='tonexty', fillcolor='rgba(200,200,200,0.3)',
    line=dict(color='lightgrey'), showlegend=False
))

# Línea de pronóstico
fig.add_trace(go.Scatter(
    x=df_fc_gen.index, y=df_fc_gen['pred_generacion'],
    mode='lines', line=dict(dash='dash'), name='Pronóstico (30 días)'
))

fig.update_layout(
    title="Generación Solar: Últimos 30 días vs Próximos 30 días",
    xaxis_title="Fecha",
    yaxis_title=gen_col.replace('_',' ').title(),
    width=800,
    height=400
)

fig.show()


En cuanto al pronostico hacia fechas futuras (30 días adelante) se comporta de la misma forma, con pronosticos que se mantienen sin una mayor variación.

Comprobamos los supuestos del modelo.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox

def check_model_assumptions(residuals, model_type, lags=12):
    """
    Comprueba supuestos de un SARIMAX/ARIMA.
    residuals: pd.Series de residuos.
    model_type: 'arima' (ignorar VAR aquí).
    lags: número de retardos para Ljung–Box y ARCH‐LM.
    Devuelve DataFrame con Test, Estadístico, p-valor y Cumple.
    """
    rows = []
    # 1) Autocorrelación univariante (Ljung–Box)
    lb = acorr_ljungbox(residuals.dropna(), lags=[lags], return_df=True).iloc[0]
    rows.append({
        'Test':        f'Ljung–Box (lag={lags})',
        'Estadístico': lb['lb_stat'],
        'p-valor':     lb['lb_pvalue'],
        'Cumple':      lb['lb_pvalue'] > 0.05
    })
    # 2) Normalidad Jarque–Bera
    jb_s, jb_p, _, _ = jarque_bera(residuals.dropna())
    rows.append({
        'Test':        'Normalidad JB',
        'Estadístico': jb_s,
        'p-valor':     jb_p,
        'Cumple':      jb_p > 0.05
    })
    # 3) Heterocedasticidad ARCH‐LM
    lm_s, lm_p, _, _ = het_arch(residuals.dropna(), nlags=lags)
    rows.append({
        'Test':        'ARCH‐LM',
        'Estadístico': lm_s,
        'p-valor':     lm_p,
        'Cumple':      lm_p > 0.05
    })
    return pd.DataFrame(rows)

# --- Aplicación al SARIMAX res_gen ---
residuals_arima = res_gen.resid

table_arima = check_model_assumptions(
    residuals=residuals_arima,
    model_type='arima',
    lags=12
)

print("\n=== Supuestos Modelo SARIMAX (ARIMAX) ===")
print(table_arima.to_string(index=False))



=== Supuestos Modelo SARIMAX (ARIMAX) ===
              Test  Estadístico      p-valor  Cumple
Ljung–Box (lag=12)     9.499646 6.597648e-01    True
     Normalidad JB   209.845902 2.707331e-46   False
           ARCH‐LM    53.582975 3.243412e-07   False


Con un nivel de significación del 0.05:

- En este caso se indica que no existe autocorrelación significativa en los residuos. Es decir, el modelo ha capturado bien la estructura temporal de la generación energética.

- Los residuos del modelo no siguen una distribución normal. (posiblemente dado por la distribución asimétrica de las variables)

- También no se logra conseguir una varianza constante en los residuos. Significando que la varianza cambia a lo largo de la serie, afectando al ajuste y pronósticos del modelo, y con ello a la precisión de los intervalos de confianza (lo cual tampoco es válido por el incumplimiento del supuesto de normalidad).



Ahora revisamos sus metricas (RMSE y MAE).

In [ ]:
# 5.b) Crear tabla de métricas
metrics_df = pd.DataFrame({
    'Métrica': ['RMSE', 'MAE'],
    'Valor':   [rmse, mae]
})

print("\n=== Métricas de validación SARIMAX (20%) ===")
print(metrics_df.to_string(index=False))



=== Métricas de validación SARIMAX (20%) ===
Métrica      Valor
   RMSE 458.049046
    MAE 374.315194


En cuanto a la predicción de la generación de energía solar los errores son altos. Donde en promedio (MAE) el ajuste genera pronósticos que se desvían 374 MWh al día en promedio. Donde, también existen momentos en los cuales son mayores (RMSE>MAE).

Debido a las métricas analizadas se puede determinar que los errores elevados sugieren que el modelo de generación de energía con variable exógena temperatura máxima no capta los movimientos de manera adecuada, y viendo los gráficos de validación también se indica que se subestima consistentemente la variabilidad de la generación de energía. La temperatura máxima resulta insuficiente para el pronóstico adecuado de la generación diaria de energía.

Comprobamos ahora un sarimax para generación de energía solar con la presión promedio.

In [ ]:
import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
warnings.filterwarnings("ignore")
# --- 0) Detectar columnas relevantes ---
# Encuentra la columna de generación en df_solar (que contenga 'generacion' en su nombre)
gen_col = next(c for c in df_solar.columns if 'generacion' in c.lower())
exo_col = 'presion_promedio'

# --- 1) Prepara índices datetime ---
df_s = df_solar.copy()
if not isinstance(df_s.index, pd.DatetimeIndex):
    if 'fecha_dia' in df_s.columns:
        df_s['fecha_dia'] = pd.to_datetime(df_s['fecha_dia'])
        df_s.set_index('fecha_dia', inplace=True)
    else:
        raise KeyError("df_solar no tiene índice datetime ni columna 'fecha_dia'")

df_m = df_meteorologicas.copy()
if not isinstance(df_m.index, pd.DatetimeIndex):
    if 'fecha_dia' in df_m.columns:
        df_m['fecha_dia'] = pd.to_datetime(df_m['fecha_dia'])
        df_m.set_index('fecha_dia', inplace=True)
    else:
        raise KeyError("df_meteorologicas no tiene índice datetime ni columna 'fecha_dia'")

# --- 2) Merge inner en fechas comunes ---
df = df_s[[gen_col]].join(df_m[[exo_col]], how='inner')

# --- 3) Define target y exógena ---
y    = df[gen_col]
exog = df[[exo_col]]

# --- 4) Grid‐search SARIMAX(p,d,q)(P,D,Q,7) ---
s = 7
p_rng = range(0, 3)
d_rng = range(0, 2)
q_rng = range(0, 3)
P_rng = range(0, 2)
D_rng = range(0, 2)
Q_rng = range(0, 2)

best_aic      = np.inf
best_order    = None
best_seasonal = None

for p, d, q in itertools.product(p_rng, d_rng, q_rng):
    for P, D, Q in itertools.product(P_rng, D_rng, Q_rng):
        try:
            m = SARIMAX(
                y,
                exog=exog,
                order=(p, d, q),
                seasonal_order=(P, D, Q, s),
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            r = m.fit(disp=False)
            if r.aic < best_aic:
                best_aic      = r.aic
                best_order    = (p, d, q)
                best_seasonal = (P, D, Q, s)
        except:
            continue

print(f"Mejor SARIMAX encontrado: order={best_order}, seasonal_order={best_seasonal}, AIC={best_aic:.2f}")



Mejor SARIMAX encontrado: order=(2, 1, 2), seasonal_order=(0, 1, 1, 7), AIC=7589.23


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go
warnings.filterwarnings("ignore")

# --- Supuestos previos: ya tienes en el entorno ---
# df               : DataFrame con índice datetime y columnas ['generacion...', 'temperatura_maxima']
# best_order       : tu tupla óptima (p,d,q)
# best_seasonal    : tu tupla óptima (P,D,Q,s)

gen_col = next(c for c in df.columns if 'generacion' in c.lower())
exo_col = 'presion_promedio'

# 1) Split 80% / 20%
n        = len(df)
n_train  = int(n * 0.8)
train    = df.iloc[:n_train]
validate = df.iloc[n_train:]

# 2) Ajustar SARIMAX solo con entrenamiento
mod_val = SARIMAX(
    train[gen_col],
    exog=train[[exo_col]],
    order=best_order,
    seasonal_order=best_seasonal,
    enforce_stationarity=False,
    enforce_invertibility=False
)
res_val = mod_val.fit(disp=False)

# 3) Pronóstico sobre validación
steps = len(validate)
pred   = res_val.get_forecast(steps=steps, exog=validate[[exo_col]])
forecast_mean = pred.predicted_mean

# 4) Construir DataFrame comparativo
df_val = validate[[gen_col]].copy()
df_val['pred_generacion'] = forecast_mean.values

# 5) Métricas de validación
rmse = np.sqrt(mean_squared_error(df_val[gen_col], df_val['pred_generacion']))
mae  = mean_absolute_error(df_val[gen_col], df_val['pred_generacion'])
print(f"Validación SARIMAX: RMSE = {rmse:.3f}, MAE = {mae:.3f}")

# 6) Gráfico comparativo
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_val.index, y=df_val[gen_col],
    mode='lines', name='Real'
))
fig.add_trace(go.Scatter(
    x=df_val.index, y=df_val['pred_generacion'],
    mode='lines', line=dict(dash='dash'), name='Predicho'
))
fig.update_layout(
    title="Validación SARIMAX: Real vs Predicho",
    title_x=0.5,
    xaxis_title="Fecha",
    yaxis_title=gen_col.replace('_',' ').title(),
    width=800, height=400
)
fig.show()

# 7) El DataFrame final df_val incluye fecha, real y predicho:
print(df_val.head())


Validación SARIMAX: RMSE = 420.536, MAE = 337.922


            generacion_MHw_sol_desierto  pred_generacion
fecha_dia                                               
2025-03-14                      1252.79       710.568490
2025-03-15                      1178.41       992.068126
2025-03-16                       950.99       926.325308
2025-03-17                       867.23       983.078794
2025-03-18                      1375.90       992.485832


In [ ]:
import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go
warnings.filterwarnings("ignore")

# --- 0) Asegura índices datetime en ambos DataFrames ---
def ensure_datetime_index(df, col='fecha_dia'):
    df = df.copy()
    if not isinstance(df.index, pd.DatetimeIndex):
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        else:
            raise KeyError(f"{df.name} necesita columna '{col}' o índice datetime")
    return df.sort_index().asfreq('D').fillna(method='ffill')

df_s = ensure_datetime_index(df_solar)
df_s.name = 'df_solar'
df_m = ensure_datetime_index(df_meteorologicas)
df_m.name = 'df_meteorologicas'

# --- 1) Merge para entrenamiento: solo fechas comunes ---
# Detecta columnas
gen_col = next(c for c in df_s.columns if 'generacion' in c.lower())
exo_var = 'presion_promedio'

df_train = df_s[[gen_col]].join(df_m[[exo_var]], how='inner')

# --- 2) Grid‐search SARIMA para la exógena ---
temp = df_m[exo_var]  # serie completa de temperatura
s = 7
p_rng = range(0,3); d_rng = range(0,2); q_rng = range(0,3)
P_rng = D_rng = Q_rng = range(0,2)

best_aic_temp = np.inf
for p,d,q in itertools.product(p_rng,d_rng,q_rng):
    for P,D,Q in itertools.product(P_rng,D_rng,Q_rng):
        try:
            mod = SARIMAX(temp,
                          order=(p,d,q),
                          seasonal_order=(P,D,Q,s),
                          enforce_stationarity=False,
                          enforce_invertibility=False)
            res = mod.fit(disp=False)
            if res.aic < best_aic_temp:
                best_aic_temp = res.aic
                best_order_temp = (p,d,q)
                best_seasonal_temp = (P,D,Q,s)
        except:
            pass

# Ajuste final de temperatura
mod_temp = SARIMAX(temp,
                   order=best_order_temp,
                   seasonal_order=best_seasonal_temp,
                   enforce_stationarity=False,
                   enforce_invertibility=False)
res_temp = mod_temp.fit(disp=False)

# Forecast 30d de la exógena
n_steps = 30
fc_temp = res_temp.get_forecast(steps=n_steps)
idx_future = pd.date_range(start=temp.index[-1] + pd.Timedelta(days=1),
                           periods=n_steps, freq='D')
df_exog_fc = fc_temp.predicted_mean.rename(exo_var).to_frame()

# --- 3) Ajuste SARIMAX de generación sobre datos comunes ---
y     = df_train[gen_col]
exog_h= df_train[[exo_var]]

# Grid‐search SARIMAX(p,d,q)(P,D,Q,7) con exógena para generación
best_aic = np.inf
for p,d,q in itertools.product(p_rng,d_rng,q_rng):
    for P,D,Q in itertools.product(P_rng,D_rng,Q_rng):
        try:
            m = SARIMAX(y,
                        exog=exog_h,
                        order=(p,d,q),
                        seasonal_order=(P,D,Q,s),
                        enforce_stationarity=False,
                        enforce_invertibility=False)
            r = m.fit(disp=False)
            if r.aic < best_aic:
                best_aic     = r.aic
                best_order   = (p,d,q)
                best_seasonal= (P,D,Q,s)
        except:
            pass

# Ajuste final de generación
mod_gen = SARIMAX(y,
                  exog=exog_h,
                  order=best_order,
                  seasonal_order=best_seasonal,
                  enforce_stationarity=False,
                  enforce_invertibility=False)
res_gen = mod_gen.fit(disp=False)

# --- 4) Pronóstico generación 30d usando exógenas predichas ---
forecast = res_gen.get_forecast(steps=n_steps, exog=df_exog_fc)
pred_mean = forecast.predicted_mean
conf_int  = forecast.conf_int()

df_fc_gen = pd.DataFrame({
    'pred_generacion': pred_mean,
    'lower_ci':        conf_int.iloc[:, 0],
    'upper_ci':        conf_int.iloc[:, 1],
    exo_var:           df_exog_fc[exo_var]
}, index=idx_future)

# 5) Mostrar resultados y gráfico con 30 días de histórico y 30 días de pronóstico

# asume que ya tienes:
# y: pd.Series de generación histórica (entrenada sobre todo el histórico común)
# df_fc_gen: DataFrame con índice de 30 días futuros y columnas ['pred_generacion','lower_ci','upper_ci',exo_var]

import plotly.graph_objects as go

# Extrae los últimos 30 días de histórico
hist_last30 = y.iloc[-30:]

fig = go.Figure()

# Histórico (últimos 30 días)
fig.add_trace(go.Scatter(
    x=hist_last30.index, y=hist_last30.values,
    mode='lines', name='Histórico (últimos 30 días)'
))

# Banda de confianza del forecast
fig.add_trace(go.Scatter(
    x=df_fc_gen.index, y=df_fc_gen['upper_ci'],
    mode='lines', line=dict(color='lightgrey'), showlegend=False
))
fig.add_trace(go.Scatter(
    x=df_fc_gen.index, y=df_fc_gen['lower_ci'],
    mode='lines', fill='tonexty', fillcolor='rgba(200,200,200,0.3)',
    line=dict(color='lightgrey'), showlegend=False
))

# Línea de pronóstico
fig.add_trace(go.Scatter(
    x=df_fc_gen.index, y=df_fc_gen['pred_generacion'],
    mode='lines', line=dict(dash='dash'), name='Pronóstico (30 días)'
))

fig.update_layout(
    title="Generación Solar: Últimos 30 días vs Próximos 30 días",
    xaxis_title="Fecha",
    yaxis_title=gen_col.replace('_',' ').title(),
    width=800,
    height=400
)

fig.show()


In [ ]:
# Suponiendo que ya has definido la función check_model_assumptions(residuals, model_type, lags)

# 1) Extrae los residuos de tu ajuste SARIMAX
residuals_arima = res_gen.resid

# 2) Comprueba los supuestos con la función
table_sarimax = check_model_assumptions(
    residuals=residuals_arima,
    model_type='arima',
    lags=12
)

# 3) Muestra la tabla
print("\n=== Supuestos Modelo SARIMAX (ARIMAX) ===")
print(table_sarimax.to_string(index=False))



=== Supuestos Modelo SARIMAX (ARIMAX) ===
              Test  Estadístico      p-valor  Cumple
Ljung–Box (lag=12) 1.308135e+02 4.237492e-22   False
     Normalidad JB 1.221093e+06 0.000000e+00   False
           ARCH‐LM 9.048584e+01 3.975691e-14   False


In [ ]:
# 5.b) Crear tabla de métricas
metrics_df = pd.DataFrame({
    'Métrica': ['RMSE', 'MAE'],
    'Valor':   [rmse, mae]
})

print("\n=== Métricas de validación SARIMAX (20%) ===")
print(metrics_df.to_string(index=False))



=== Métricas de validación SARIMAX (20%) ===
Métrica     Valor
   RMSE 420.53641
    MAE 337.92209


Con el ajuste del modelo SARIMAX(2, 1, 2)(0, 1, 1)[7] para generación de energía, pero esta vez como variable exogena presion_promedio pronosticada por VAR(7). Si la comparamos la que se utilizó temperatura_maxima como exogena con este ajuste, el modelo ajustado con presión promedio entregó un menor RMSE (420 MW por día) y un promedio de error MAE (337) menor igualmente.

Coeficientes de ambos modelos SARIMAX

In [ ]:
# --- 6) Mostrar coeficientes de los dos modelos ajustados ---

# Coeficientes del modelo SARIMA de la exógena (temperatura_maxima)
coef_temp = res_temp.params.reset_index()
coef_temp.columns = ['Parámetro', 'Valor']
print("\nCoeficientes del SARIMA (exógena: temperatura_maxima):")
print(coef_temp.to_string(index=False))

# Coeficientes del modelo SARIMAX de generación (generacion_MWh_sol_desierto)
coef_gen = res_gen.params.reset_index()
coef_gen.columns = ['Parámetro', 'Valor']
print("\nCoeficientes del SARIMAX (generación con exógena):")
print(coef_gen.to_string(index=False))



Coeficientes del SARIMA (exógena: temperatura_maxima):
Parámetro     Valor
    ar.L1  0.557272
    ma.L1 -0.853852
    ma.L2 -0.043786
  ma.S.L7 -0.014123
   sigma2  2.748702

Coeficientes del SARIMAX (generación con exógena):
       Parámetro        Valor
presion_promedio    23.009411
           ar.L1    -0.498095
           ar.L2     0.387958
           ma.L1    -0.073356
           ma.L2    -0.846729
         ma.S.L7    -0.951740
          sigma2 94534.054653
